In [178]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import csv, json

In [180]:
ls ../RawData

24-oct-2025 Nilma Jesper/


In [181]:
input_directory = '../RawData'
output_directory = '../ChunkedData/'

In [184]:
os.listdir(input_directory)
onlydir = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
onlyfiles = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]

In [186]:
onlydir

['24-oct-2025 Nilma Jesper']

In [188]:
def parse_csv_line(line: str):
    # csv.reader handles the quoted JSON payload with doubled quotes
    return next(csv.reader([line], delimiter=',', quotechar='"', doublequote=True))

In [190]:
def is_json(perhaps_json):
  try:
    json.loads(perhaps_json)
  except ValueError as e:
    return False
  return True


In [192]:
dfs_list = []

dirs = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
count = 0 
for directory in dirs:
    for input_file in os.listdir(input_directory+'/'+directory):
        file_path = input_directory+'/'+directory+'/'
        file_name = os.path.basename(input_file)
        save_lines = False
        if file_name.endswith('.csv'):
            with open(file_path+file_name) as f:
                for line in f:
                # Do something with 'line'        
                #line = msg_line
                    
                    row =  parse_csv_line(line)
                    if row[2] == 'MESSAGE': 
                        row_dict = json.loads(row[3])
                        if is_json(row_dict['message']):
                            message = json.load(row_dict['message'])
                        else:
                            message_list = row_dict['message'].split(',')
                            message = {}
                            message['action'] = message_list[0]
                            for i in range(1, len(message_list)):
                                message['id'+str(i)] = message_list[i] 
                        print(message)
                        
                        if message['action'] =='start':
                            message_values = list(message.values())
                            file_addition = '_'.join(message_values[1:])
                            #print(file_addition)
                            outputfile = output_directory+file_addition+'_'+file_name
                            start_time = row_dict['timestamp']
                            # Add information from message
                            message.pop('action')
                            start_row = message
                            start_row['start_time'] = start_time
                            start_row['channelId'] =  row_dict['channelId']
                            
                            if(not(save_lines)):
                                f_out = open(outputfile, "w")   # 'r' for reading and 'w' for writing                       
                                save_lines = True
                            else:
                                print(outputfile)
                                f_out = open(outputfile, "w")
            
                        elif message['action'] == 'stop':
                            end_time = row_dict['timestamp']
                            duration = end_time - start_time
                            start_row['exp_duration'] = duration
                            start_row['count'] = count
                            # print(start_row)
                            count = 0
                            print('meta_row=',start_row)
                            save_lines = False # No more lines for this flow  
                            f_out.close()   # Close output file
                            # save the meta data as the first two lines of the chunked CSV file
                            a = list(start_row.keys())
                            line1 = ','.join(a) 
                            a = list(start_row.values())
                            line2 = ','.join([str(s) for s in a])
                            with open(outputfile, 'r') as original:
                                data = original.read()
                            with open(outputfile, 'w') as modified:
                                modified.write(line1+"\n"+line2+"\n" + data)
                        else:
                            count += 1
                            print(count)
                            
                                
                    elif row[2] == 'POWER' and save_lines:  
                        f_out.write(line)   # Write line into the file 



{'action': 'start', 'id1': 'sigmark', 'id2': '80'}
{'action': 'stop', 'id1': 'sigmark', 'id2': '80'}
meta_row= {'id1': 'sigmark', 'id2': '80', 'start_time': 1761304914559, 'channelId': 'CH1', 'exp_duration': 80021, 'count': 0}
{'action': 'start', 'id1': 'sigmark', 'id2': '20'}
{'action': 'sigmark', 'id1': 'tick'}
1
{'action': 'sigmark', 'id1': 'tick'}
2
{'action': 'sigmark', 'id1': 'tick'}
3
{'action': 'sigmark', 'id1': 'tick'}
4
{'action': 'stop', 'id1': 'sigmark', 'id2': '20'}
meta_row= {'id1': 'sigmark', 'id2': '20', 'start_time': 1761304994596, 'channelId': 'CH1', 'exp_duration': 80080, 'count': 4}
{'action': 'start', 'id1': 'sigmark', 'id2': '10'}
{'action': 'sigmark', 'id1': 'tick'}
1
{'action': 'sigmark', 'id1': 'tick'}
2
{'action': 'sigmark', 'id1': 'tick'}
3
{'action': 'sigmark', 'id1': 'tick'}
4
{'action': 'sigmark', 'id1': 'tick'}
5
{'action': 'sigmark', 'id1': 'tick'}
6
{'action': 'sigmark', 'id1': 'tick'}
7
{'action': 'sigmark', 'id1': 'tick'}
8
{'action': 'stop', 'id1': '